In [3]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

# Downloading model checkpoint to a local dir model_dir
model_dir = snapshot_download('Qwen/Qwen-14B-Chat-Int4', cache_dir='./')

2024-04-30 09:34:48,385 - modelscope - WARNING - Model revision not specified, use revision: v1.0.8
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 8.21k/8.21k [00:00<00:00, 11.7MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 50.8k/50.8k [00:00<00:00, 689kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1.17k/1.17k [00:00<00:00, 3.86MB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 76.0/76.0 [00:00<00:00, 287kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 2.29k/2.29k [00:00<00:00, 9.92MB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1.88k/1.88k [00:00<00:00, 7.83MB/s]
Downloading: 100%|████████████

In [35]:
from vllm import LLM
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

llm = LLM(model="Qwen/Qwen1___5-14B-Chat-AWQ/", gpu_memory_utilization=0.6)

WARNING 04-09 09:44:40 config.py:211] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 04-09 09:44:40 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='Qwen/Qwen1___5-14B-Chat-AWQ/', tokenizer='Qwen/Qwen1___5-14B-Chat-AWQ/', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-09 09:44:43 model_runner.py:104] Loading model weights took 9.0594 GB
INFO 04-09 09:44:48 gpu_executor.py:94] # GPU blocks: 839, # CPU blocks: 327


ValueError: The model's max seq len (32768) is larger than the maximum number of tokens that can be stored in KV cache (13424). Try increasing `gpu_memory_utilization` or decreasing `max_model_len` when initializing the engine.

In [8]:
from vllm import LLM, SamplingParams
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(generated_text)

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 13.29it/s]

 Wang Lei and I am from China. I am a 14-year-old
 the head of state and government of the United States. The president leads the executive
____．（　　）
A. Berlin
B. Rome
C.
 exciting， but it also raises important questions about the impact on jobs and society.


In [1]:
from langchain_community.llms import VLLM
import os

llm = VLLM(
    model="Qwen/Qwen1___5-7B-Chat/",
    trust_remote_code=True,  # mandatory for hf models
    max_new_tokens=2560,
    top_k=10,
    top_p=0.95,
    temperature=0.8,
)

INFO 04-08 15:20:30 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='Qwen/Qwen1___5-7B-Chat/', tokenizer='Qwen/Qwen1___5-7B-Chat/', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-08 15:20:32 selector.py:51] Cannot use FlashAttention because the package is not found. Please install it for better performance.
INFO 04-08 15:20:32 selector.py:25] Using XFormers backend.
INFO 04-08 15:20:39 model_runner.py:104] Loading model weights took 14.3919 GB
INFO 04-08 15:20:42 gpu_executor.py:94] # GPU blocks: 2264, # CPU blocks: 512
INFO 04-08 15:20:44 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-08 15:20:44 model_runner.py:795] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-08 15:20:49 model_runner.py:867] Graph capturing finished in 5 secs.


In [19]:
!pwd

/data/liucd/langchain


In [8]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1,5'

In [18]:
# 量化版本
!python -m vllm.entrypoints.openai.api_server --served-model-name Qwen1.5-32B-Chat-GPTQ \
      --model Qwen/Qwen-72B-Chat-Int4/  --port 1122 --quantization gptq \
      --gpu-memory-utilization 0.8 --tensor-parallel-size 2 --trust-remote-code   --max-model-len 4096 

INFO 05-22 15:10:42 api_server.py:151] vLLM API server version 0.4.1
INFO 05-22 15:10:42 api_server.py:152] args: Namespace(host=None, port=1122, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, served_model_name=['Qwen1.5-32B-Chat-GPTQ'], lora_modules=None, chat_template=None, response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], model='Qwen/Qwen-72B-Chat-Int4/', tokenizer=None, skip_tokenizer_init=False, revision=None, code_revision=None, tokenizer_revision=None, tokenizer_mode='auto', trust_remote_code=True, download_dir=None, load_format='auto', dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, max_model_len=4096, guided_decoding_backend='outlines', worker_use_ray=False, pipeline_parallel_size=1, tensor_parallel_size=2, max_parallel_loading_workers=None, ray_workers_use_nsight=False, block_size=16, enable_

In [13]:
vllm.__version__

NameError: name 'vllm' is not defined

In [4]:
# 非量化版本: 7B 模型
!python -m vllm.entrypoints.openai.api_server --served-model-name Qwen1.5-7B-Chat \
      --model Qwen/Qwen1___5-14B-Chat  --port 1122

usage: api_server.py [-h] [--host HOST] [--port PORT]
                     [--uvicorn-log-level {debug,info,warning,error,critical,trace}]
                     [--allow-credentials] [--allowed-origins ALLOWED_ORIGINS]
                     [--allowed-methods ALLOWED_METHODS]
                     [--allowed-headers ALLOWED_HEADERS] [--api-key API_KEY]
                     [--served-model-name SERVED_MODEL_NAME]
                     [--lora-modules LORA_MODULES [LORA_MODULES ...]]
                     [--chat-template CHAT_TEMPLATE]
                     [--response-role RESPONSE_ROLE]
                     [--ssl-keyfile SSL_KEYFILE] [--ssl-certfile SSL_CERTFILE]
                     [--ssl-ca-certs SSL_CA_CERTS]
                     [--ssl-cert-reqs SSL_CERT_REQS] [--root-path ROOT_PATH]
                     [--middleware MIDDLEWARE] [--model MODEL]
                     [--tokenizer TOKENIZER] [--revision REVISION]
                     [--code-revision CODE_REVISION]
                     

In [3]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:1122/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

chat_response = client.chat.completions.create(
    model="Qwen1.5-7B-Chat",
    messages=[
        {"role": "system", "content": "你是一个人工智能助手."},
        {"role": "user", "content": "告诉我有关北京的特产"},
    ]
)
print("Chat response:", chat_response)

APIConnectionError: Connection error.